# agentscope.model

提供了一套统一的异步聊天模型接口，支持多种主流大模型平台（如 OpenAI、Anthropic、DashScope、Gemini、Ollama），并原生支持：

- 工具调用（Function Calling）
- 结构化输出（Structured Output via Pydantic）
- 多模态内容（通过 Message 模块）
- 流式响应（Streaming）

核心组件有
- `ChatModelBase`：所有聊天模型的抽象基类  
- `ChatResponse`：模型返回的统一响应格式  
- 具体实现类：
  - `OpenAIChatModel`：封装 OpenAI API  
  - `AnthropicChatModel`：封装 Anthropic（Claude）API  
  - `DashScopeChatModel`：封装阿里通义千问（Qwen）API  
  - `GeminiChatModel`：封装 Google Gemini API  
  - `OllamaChatModel`：封装本地 Ollama 服务 

## ChatResponse：统一响应格式
字段说明
- content: List[TextBlock | ToolUseBlock | ThinkingBlock]
支持文本、工具调用、推理过程
- id: 响应唯一 ID
- created_at: 时间戳
- usage: 可选的 token 使用统计（如 {"prompt_tokens": 50, "completion_tokens": 30}）
> 注意：ChatResponse.content 是 内容块列表，不是字符串！ 

In [9]:
from agentscope.model import DashScopeChatModel
from dotenv import load_dotenv
import os
load_dotenv()

model = DashScopeChatModel(
    model_name="qwen-vl-plus",
    api_key=os.environ["DASHSCOPE_API_KEY"],
    stream=False
)

# 多模态消息：文本 + 图片（修正格式）
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "text", 
                "text": "图中是什么"
            },
            {
                "type": "image", 
                "image": "https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen-VL/assets/demo.jpeg"
            }
        ]
    }
]

response = await model(messages=messages)

for block in response.content:
    if hasattr(block, 'text'):  # TextBlock 有 text 属性
        print("Answer:", block.text)
    elif isinstance(block, dict) and block.get("type") == "text":
        print("Answer:", block["text"])


Answer: 这张图片展示了一位女性和一只狗在海滩上的温馨场景。以下是图片的详细描述：

1. **人物**：
   - 一位女性坐在沙滩上，面带微笑，显得非常开心和放松。
   - 她穿着一件格子衬衫和牛仔裤，脚上没有穿鞋，直接踩在沙子上。

2. **动物**：
   - 一只黄色的狗（可能是拉布拉多犬）坐在女性的旁边。
   - 狗戴着一个彩色的胸背带，看起来很健康和友好。
   - 狗的前爪搭在女性的手上，似乎正在进行“握手”的动作，表现出人与动物之间的亲密互动。

3. **环境**：
   - 场景是在海滩上，沙滩细腻柔软，背景是广阔的海洋。
   - 海浪轻轻拍打着岸边，远处的地平线与天空相接，给人一种宁静和开阔的感觉。
   - 太阳的位置较低，可能是傍晚时分，阳光洒在沙滩和人物身上，营造出温暖而柔和的光线效果。

4. **氛围**：
   - 整个画面充满了温馨、和谐和快乐的氛围。
   - 人与动物之间的互动显得非常自然和亲密，仿佛在享受一段美好的时光。

总结来说，这张图片捕捉了一个充满爱与欢乐的瞬间，展现了人与宠物之间的深厚情感，同时也让人感受到大自然的美好与宁静。


In [ ]:
# 流式输出
from agentscope.model import DashScopeChatModel
from dotenv import load_dotenv
import os
load_dotenv()

model = DashScopeChatModel(
    model_name="qwen-vl-plus",
    api_key=os.environ["DASHSCOPE_API_KEY"],
)

# 多模态消息：文本 + 图片（修正格式）
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "text", 
                "text": "图中是什么"
            },
            {
                "type": "image", 
                "image": "https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen-VL/assets/demo.jpeg"
            }
        ]
    }
]


response_stream = await model(messages=messages)

async for response in response_stream:
    for block in response.content:
        if hasattr(block, 'text'):  # TextBlock 有 text 属性
            print(block.text, end='', flush=True)  # 不换行，实时输出
        elif isinstance(block, dict) and block.get("type") == "text":
            print(block["text"], end='', flush=True)


这张这张图片这张图片展示这张图片展示了一这张图片展示了一位这张图片展示了一位女性这张图片展示了一位女性和这张图片展示了一位女性和一只这张图片展示了一位女性和一只狗这张图片展示了一位女性和一只狗在这张图片展示了一位女性和一只狗在海滩这张图片展示了一位女性和一只狗在海滩上的这张图片展示了一位女性和一只狗在海滩上的温馨这张图片展示了一位女性和一只狗在海滩上的温馨场景这张图片展示了一位女性和一只狗在海滩上的温馨场景。这张图片展示了一位女性和一只狗在海滩上的温馨场景。以下是这张图片展示了一位女性和一只狗在海滩上的温馨场景。以下是图片这张图片展示了一位女性和一只狗在海滩上的温馨场景。以下是图片的这张图片展示了一位女性和一只狗在海滩上的温馨场景。以下是图片的详细这张图片展示了一位女性和一只狗在海滩上的温馨场景。以下是图片的详细描述这张图片展示了一位女性和一只狗在海滩上的温馨场景。以下是图片的详细描述：

这张图片展示了一位女性和一只狗在海滩上的温馨场景。以下是图片的详细描述：

1这张图片展示了一位女性和一只狗在海滩上的温馨场景。以下是图片的详细描述：

1.这张图片展示了一位女性和一只狗在海滩上的温馨场景。以下是图片的详细描述：

1. **这张图片展示了一位女性和一只狗在海滩上的温馨场景。以下是图片的详细描述：

1. **人物这张图片展示了一位女性和一只狗在海滩上的温馨场景。以下是图片的详细描述：

1. **人物**这张图片展示了一位女性和一只狗在海滩上的温馨场景。以下是图片的详细描述：

1. **人物**：
这张图片展示了一位女性和一只狗在海滩上的温馨场景。以下是图片的详细描述：

1. **人物**：
  这张图片展示了一位女性和一只狗在海滩上的温馨场景。以下是图片的详细描述：

1. **人物**：
   -这张图片展示了一位女性和一只狗在海滩上的温馨场景。以下是图片的详细描述：

1. **人物**：
   - 这张图片展示了一位女性和一只狗在海滩上的温馨场景。以下是图片的详细描述：

1. **人物**：
   - 一位这张图片展示了一位女性和一只狗在海滩上的温馨场景。以下是图片的详细描述：

1. **人物**：
   - 一位女性这张图片展示了一位女性和一只狗在海滩上的温馨场景。以下是图片的详细描述：

1. **人物**：
   - 一位女性坐在这张图片